# Calculating NBA Draft Probabilities

### Introduction:
The fourteen NBA teams that did not make the playoffs participate in the lottery.  Fourteen ping pong balls are placed in a lottery machine, and each draft pick is determined by drawing four balls.  There are 1,000 valid combinations of drawings.  For example, one drawing is 11-3-5-8.  The fourteen teams are seeded based on their regular season records and they are each assigned a certain number of combinations.  Seeds 1-3, the teams with the worst records, are each assigned 140 combinations.  The remaining teams are assigned fewer combinations.  Therefore, the probability that any of the worst three teams receive the the number 1 pick is 140/1000 or 14%.  The top four picks are determined by drawing four combinations.  Afterwards, picks 5-14 are assigned to teams according to their seedings.  See this wikipedia page for a more in-depth explanation of the draft: https://en.wikipedia.org/wiki/NBA_draft_lottery#Process

### Goal:
My  goal is to calculate probabilities that the seeds 1-14 receive any pick.  These probabilities are listed on the wikipedia page linked above.  


### Calculating First Pick Probabilities:
Divide each team's number of combinations by 1000.  The probability seed 1 gets pick 1 is then $P(1\rightarrow1)=140/1000$.

### Calculating Second Pick Probabilities:
First denote the probability that team i gets the first pick as $f_i$.  We calculated this above. Let's write a formula for calculating the probability that seed 1 gets pick 2.  Seed 1 can only get pick 2 if one of seeds 2-14 gets seed 1, so we calculate:

$P(1\rightarrow2)=\Sigma_{i\neq1}^{14}P(i\rightarrow1\cap1\rightarrow2)$

Let's calculate the term $P(2\rightarrow1\cap1\rightarrow2)$ in the summation above.  We have $P(2\rightarrow1\cap1\rightarrow2)=P(2\rightarrow1)P*P(1\rightarrow2|2\rightarrow1)=\frac{140}{1000}*\frac{140}{1000-140}=\frac{140}{1000}*\frac{\frac{140}{1000}}{\frac{1000-140}{1000}}=\frac{140}{1000}*\frac{\frac{140}{1000}}{1-\frac{140}{1000}}=f_1*\frac{f_2}{1-f_2}$.  

When we calculate each term in the summation above we have $P(1\rightarrow2)=f_1\Sigma_{k\neq1}^{14}\frac{f_k}{1-f_k}$.

Thus, for a seed i we have $P(i\rightarrow2)=f_i\Sigma_{k\neq i}^{14}\frac{f_k}{1-f_k}$
 
### Calculating Third Pick Probabilities:
We use similar steps to arrive at $P(i\rightarrow3)=f_i\Sigma_{k\neq i}^{14}\Sigma_{l\neq i}^{14}\frac{f_k}{1-f_k}*\frac{f_l}{1-f_k-f_l}$

### Calculating Fourth Pick Probabilities:
We use similar steps to arrive at $P(i\rightarrow4)=f_i\Sigma_{k\neq i}^{14}\Sigma_{l\neq i}^{14}\Sigma_{m\neq i}^{14}\frac{f_k}{1-f_k}*\frac{f_l}{1-f_k-f_l}*\frac{f_m}{1-f_k-f_l-f_m}$.

### Calculating 5-14 Pick Probabilities:
If we know the seeds that get the first 4 picks, then we instantly know which seeds get picks 5-14.  Let's say we want to calculate the probability that the 1st seed gets the 5th pick- this is equivalent to the probability of any other 4 teams getting picks 1-4.  To do this, we utilize the equation above and add the conditions that $i\neq1$, $k\neq1$, $l\neq1$, and $m\neq1$.  

In [1]:
import numpy as np
import pandas as pd

In [2]:
#probabilities of each team chosen first
new_system = [.14,.14,.14,.125,.105,.09,.075,.06,.045,.03,.02,.015,.01,.005]

#14x4 matrix
#rows are teams
#columns are picks 1-4
#values are probabilities that a given team gets a given pick
picks1_4 = np.zeros((14,4))

#set first column equal to chances that each team gets first pick
picks1_4[:,0] = new_system

In [3]:
#calculate probabilities of each seed chosen second
#iterate through teams chosen second
for j in range(len(new_system)):
    prob = 0
    seed = new_system[j]
    #iterate through teams chosen first
    for i in range(len(new_system)):
        if i!=j:
            prob = prob + new_system[j]*(new_system[i]/(1-new_system[i]))
    
    picks1_4[j,1]=prob

In [4]:
# calculate probabilities for each seed chosen third
for j in range(len(new_system)):
    prob = 0
    seed = new_system[j]
    for i in range(len(new_system)):
        for l in range(len(new_system)):
            if (i!=j) and (i!=l) and (l!=j):
                prob = prob + new_system[j]*(new_system[i]/(1-new_system[i]))*(new_system[l]/(1-new_system[i]-new_system[l]))
    picks1_4[j,2]=prob

In [5]:
# calculate probabilities for each seed chosen fourth
for j in range(len(new_system)):
    prob = 0
    seed = new_system[j]
    for i in range(len(new_system)):
        for l in range(len(new_system)):
            for z in range(len(new_system)):
                if (i!=j) and (j!=l) and (i!=l) and (j!=z) and (i!=z) and (l!=z):
                    prob = prob + new_system[j]*(new_system[i]/(1-new_system[i]))*(new_system[l]/(1-new_system[i]-new_system[l]))*(new_system[z]/(1-new_system[i]-new_system[l]-new_system[z]))
    picks1_4[j,3]=prob

In [6]:
#14x10 matrix
#rows are teams 1-14
#columns are picks 5-14
#values are number of lower seeds that need to get a top 4 pick...
#in order for the given seed to get the given pick
matrix = [[4,None,None,None,None,None,None,None,None,None],
         [3,4,None,None,None,None,None,None,None,None],
         [2,3,4,None,None,None,None,None,None,None],
         [1,2,3,4,None,None,None,None,None,None],
         [0,1,2,3,4,None,None,None,None,None],
         [None,0,1,2,3,4,None,None,None,None],
         [None,None,0,1,2,3,4,None,None,None],
         [None,None,None,0,1,2,3,4,None,None],
         [None,None,None,None,0,1,2,3,4,None],
         [None,None,None,None,None,0,1,2,3,4],
         [None,None,None,None,None,None,0,1,2,3],
         [None,None,None,None,None,None,None,0,1,2],
         [None,None,None,None,None,None,None,None,0,1],
         [None,None,None,None,None,None,None,None,None,0]]

In [7]:
#calculate probabilities of each seed chosen 5-14

#14x10 matrix
#rows are teams 1-14
#columns are picks 5-14
#values are probabilities that a given team gets a given pick
picks5_14 = np.zeros((14,10))

#code below populates picks5_14
for a in range(14):
    for b in range(10):
        intermediate_array=np.zeros((14))
        for num in range(0,5):
            if matrix[a][b] == num:
                for j in range(len(new_system)):
                    prob = 0
                    seed = new_system[j]
                    for i in range(len(new_system)):
                        for l in range(len(new_system)):
                            for z in range(len(new_system)):
                                if (i!=j) and (j!=l) and (i!=l) and (j!=z) and (i!=z) and (l!=z) and (j!=a) and (i!=a) and (l!=a) and (z!=a) and (sum([j>a,i>a,l>a,z>a])==num):
                                    prob = prob + new_system[j]*(new_system[i]/(1-new_system[i]))*(new_system[l]/(1-new_system[i]-new_system[l]))*(new_system[z]/(1-new_system[i]-new_system[l]-new_system[z]))
                    intermediate_array[j]=prob
 
        picks5_14[a][b] = sum(intermediate_array)

In [8]:
all_probabilities = np.concatenate((picks1_4,picks5_14),axis=1)

In [9]:
#probabilities should sum to 1 across both axis
np.sum(all_probabilities,axis=0)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [10]:
np.sum(all_probabilities,axis=1)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [11]:
seeds=["Seed"+str(i) for i in range(1,15)]
picks=["Pick" + str(i) for i in range(1,15)]
df = pd.DataFrame(all_probabilities,seeds,picks)

### The table below shows the probability that each team gets each pick

In [12]:
df

,Pick1,Pick2,Pick3,Pick4,Pick5,Pick6,Pick7,Pick8,Pick9,Pick10,Pick11,Pick12,Pick13,Pick14
Seed1,0.140,0.134173,0.127487,0.119720,0.478620,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
Seed2,0.140,0.134173,0.127487,0.119720,0.278394,0.200226,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
Seed3,0.140,0.134173,0.127487,0.119720,0.148386,0.260016,0.070218,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
Seed4,0.125,0.122289,0.118909,0.114621,0.072434,0.257408,0.167432,0.021907,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
Seed5,0.105,0.105404,0.105555,0.105281,0.022166,0.196105,0.267447,0.086845,0.006197,0.000000,0.000000,0.000000,0.000000,0.000000e+00
Seed6,0.090,0.092004,0.094087,0.096180,0.000000,0.086244,0.297661,0.205496,0.036835,0.001492,0.000000,0.000000,0.000000,0.000000e+00
Seed7,0.075,0.078007,0.081378,0.085165,0.000000,0.000000,0.197243,0.341095,0.128807,0.013008,0.000296,0.000000,0.000000,0.000000e+00
Seed8,0.060,0.063440,0.067446,0.072168,0.000000,0.000000,0.000000,0.344657,0.320976,0.067470,0.003796,0.000046,0.000000,0.000000e+00
Seed9,0.045,0.048332,0.052312,0.057153,0.000000,0.000000,0.000000,0.000000,0.507184,0.258983,0.030121,0.000909,0.000005,0.000000e+00
Seed10,0.030,0.032707,0.036004,0.040109,0.000000,0.000000,0.000000,0.000000,0.000000,0.659047,0.189910,0.012044,0.000178,3.885362e-07
